In [2]:
from danial import model

In [3]:
mod = model.Model()

/Users/dania/code/fyp/MHNet/310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dania/code/fyp/MHNet/310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
mod.summary()

Model Components:
- VGG Backbone
- Selective Weighted Attention (SPG)
- F3 Convolution
- PRM (Perception Refinement Module)
- PreFCS
- CRFCS (Conditional Random Field based Camouflage Segmentation)
